<a href="https://colab.research.google.com/github/DeanIA/dl4m_final/blob/main/map_widget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Map Widget

## Set-up notebook

Once per colab session, run this:

In [ ]:
!pip install -q ipyleaflet ipywidgets

Import Libraries

In [ ]:
# import libraries

from ipyleaflet import Map, Marker
import ipywidgets as widgets
from IPython.display import display
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from geopy.geocoders import Nominatim
from geopy.geocoders import OpenCage


## Set the mapping tokens

These are Chris's, so please consider using your own by generating API tokens on Mapbox and OpenCage.



In [ ]:
mapbox_token = 'pk.eyJ1IjoiY3A0MTAxIiwiYSI6ImNtOXJxMTU4bDA0eDIyanExNmYwcnY0NmUifQ.Oho3e4Abhdqw48TyzzNuXg'

opencage_key = '00a93009ec71466d85a55ee81d97b702'

## Load the model

If you are on Colab, run this:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/dl4m_final_project/documents/rural_suburban_urban_classifier.keras')

## Generate the map

Some helper functions to:


*   download the image at specified lat/lon
*   generate predictions for the image



In [ ]:
def download_satellite_image(lat, lon, zoom=16, size=(224, 224), access_token=None):
    if access_token is None:
        raise ValueError('Mapbox access token required.')

    url = f'https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/{lon},{lat},{zoom}/{size[0]}x{size[1]}?access_token={access_token}'
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def predict_image(img, model):
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)
    pred_label = np.argmax(pred)
    label_map = {0: 'rural', 1: 'suburban', 2: 'urban'}
    return label_map[pred_label], pred[0]

Set up the interactive map UI

In [ ]:
# default location is NYC

center = (40.7128, -74.0060)
m = Map(center=center, zoom=10)
marker = Marker(location=center, draggable=False)
m.add_layer(marker)

lat_display = widgets.FloatText(value=center[0], description='Latitude')
lon_display = widgets.FloatText(value=center[1], description='Longitude')
address_input = widgets.Text(value='', placeholder='Enter address here', description='Search:')
search_button = widgets.Button(description='Go')
predict_button = widgets.Button(description='Predict')
output = widgets.Output()

# handle map click updates

def handle_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        marker.location = coords
        lat_display.value = coords[0]
        lon_display.value = coords[1]

m.on_interaction(handle_map_click)

# address search logic

def on_search_clicked(b):
    geolocator = OpenCage(api_key=opencage_key)
    try:
        location = geolocator.geocode(address_input.value)
        if location:
            coords = (location.latitude, location.longitude)
            m.center = coords
            marker.location = coords
            lat_display.value = coords[0]
            lon_display.value = coords[1]
        else:
            with output:
                print('Address not found.')
    except Exception as e:
        with output:
            print('Geocoding error:', e)

search_button.on_click(on_search_clicked)

# display interface

display(m, address_input, search_button, lat_display, lon_display, predict_button, output)

# run prediction on click
def on_predict_clicked(b):
    output.clear_output()
    with output:
        try:
            lat, lon = lat_display.value, lon_display.value
            img = download_satellite_image(lat, lon, access_token=mapbox_token)
            label, probs = predict_image(img, model)

            plt.imshow(img)
            plt.axis('off')
            plt.title(f'Prediction: {label}')
            plt.show()

            labels = ['rural', 'suburban', 'urban']
            for i, p in enumerate(probs):
                print(f'{labels[i]}: {p:.3f}')
        except Exception as e:
            print('Error:', e)

predict_button.on_click(on_predict_clicked)

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Text(value='', description='Search:', placeholder='Enter address here')

Button(description='Go', style=ButtonStyle())

FloatText(value=40.7128, description='Latitude')

FloatText(value=-74.006, description='Longitude')

Button(description='Predict', style=ButtonStyle())

Output()